In [1]:
import torch

In [4]:
def lyapunov_binary_loss(p, Y, N, weights=None, eps=1e-6):
    """Lyapunov Central Limit loss for binary outcomes.
    
    Args:
        p (np.ndarray): Array of probabilities of shape (n, k).
        Y (int): Number of successes.
        N (int): Size of group.
    
    Returns:
        float: a normal approximation to the combinatorial log-likelihood.
    """
    # N = p.shape[0]
    # max_p = N / (N + 1)
    # p = np.clip(p, 1 - max_p, max_p)
    phi2 = torch.clamp(torch.sum(p * (1 - p), axis=0), min=eps)
    mu = torch.sum(p, axis=0)
    logp = 1/2 * torch.log(phi2) - (1 / phi2) * (Y - mu)**2
    if weights is not None:
        logp = weights * logp
    return -logp

In [5]:
import os
from elrpy.data.data import load
year = 2016
state = "AR"

save_dir = f"../ei/general/data/{state.lower()}/{year}/pre"

covars_path = f"{os.path.dirname(save_dir)}/covars.csv.zip"
results_path = f"{save_dir}/results.csv.zip"

group_data = load(
	covars_path, results_path
)

Loading data from ../ei/general/data/ar/2016/pre/results.covars.npz.
	Set force_from_csv=True to reload the data using updated files or group_cols/Y_cols/N_cols.


In [6]:
def binary_model(model_params, X):
    """Returns the sigmoid of the linear model.

    Args:
        model_params (np.ndarray): model parameters
        X (np.ndarray): data matrix

    Returns:
        np.ndarray: sigmoid of the linear model
    """
    return torch.sigmoid(X @ model_params)

In [7]:
import numpy as np
for d in group_data:
    for k in d:
        y = torch.tensor(np.array(d[k]))

In [8]:
group_data2 = tuple({k: torch.tensor(np.array(d[k])).float() for k in d} for d in group_data)

In [9]:
def loss(model_params):
    losses = torch.zeros(len(group_data2[0]))
    for i, k in enumerate(group_data2[0].keys()):
        p = binary_model(model_params, group_data2[0][k])
        losses[i] = lyapunov_binary_loss(p, group_data2[1][k], group_data2[2][k])
    return losses.mean()

In [10]:
model_params = torch.zeros(next(iter(group_data[0].values())).shape[1]).float()

In [13]:
from torch.func import hessian
hess_fn = torch.jit.script(hessian(loss))

RuntimeError: 
undefined value group_data2:
  File "/Users/njwfish/miniforge3/envs/elrpy/lib/python3.10/site-packages/torch/_functorch/eager_transforms.py", line 2
def loss(model_params):
    losses = torch.zeros(len(group_data2[0]))
                             ~~~~~~~~~~~ <--- HERE
    for i, k in enumerate(group_data2[0].keys()):
        p = binary_model(model_params, group_data2[0][k])


In [60]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [12]:
hess = hess_fn(model_params)

[2023-11-17 20:26:55,650] [4/0] torch._dynamo.utils: [WARNING] Unsupported: meta converter nyi with fake tensor propagation.


InternalTorchDynamoError: Cannot access data pointer of Tensor that doesn't have storage

from user code:
 
Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [51]:
torch.clamp(model_params, min=1e-6)

tensor([1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06,
        1.0000e-06, 1.0000e-06, 1.0000e-